- "Using build-in training & evaluation loops" with `model.fit() model.evaluate(), model.predict()`
- Writing your own training & evaluation loops from scratch using eager execution and the `GradientTape` object.

In general, whether you are using built-in loops or writing your own, model training & evaluation works strictly in the same way across every kind of Keras model -- Sequential models, models built with the Functional API, and models written from scratch via model subclassing

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

%tensorflow_version 2.x
import tensorflow as tf

tf.keras.backend.clear_session() # For easy reset of notebook state.

## Part 1: Using build in training & evaluation loops
When passing data to the built-in training loops of a model, you should either use **Numpy arrays* (if your data is small and fits in memory) or **tf.Dataset** objects. In the next few paragraphs, we'll user the MNIST dataset as Numpy array, in order to demonstrate how to user optimizers, losses and metrics

### API overview: a first end-to-end example
Let's consider the following model (here, we build in with the Functional API, but it could be a Sequential model or a subclassed model as well):

In [0]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = layers.Input(shape=(784, ), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predicitons')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [0]:
# Here's what the typical end-to-end workflow looks like, consisting of training,
# validation on a holdout set generated from the original training data,
# and finally evaluation on the test data:

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data (theses are Numpy arrays)
x_train = x_train.reshape(x_train.shape[0], -1).astype('float32') / 255.0
x_test = x_test.reshape(x_test.shape[0], -1).astype('float32') / 255.0

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

model.compile(optimizer=keras.optimizers.RMSprop(),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=3,
                    # Monitoring validation loss and metrics
                    # at the end of each eopch
                    validation_data=(x_val, y_val))

# The returned 'history' object holds a record
# of the loss values and metric values during training
print('\nhistory dict: ', history.history)

# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on the test data')
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc: ', results)

# Generate preditions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(x_test[:3])
print('predidctions shape: ', predictions.shape)

In [0]:
# Model definition and compile steps for later reuse

def un_compiled_model():
  inputs = keras.Input(shape=(784, ), name='digits')
  x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
  x = layers.Dense(64, activation='relu', name='dense_2')(x)
  outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
  model = keras.Model(inputs, outputs)
  return model

def get_compiled_model():
  model = un_compiled_model()
  model.compile(optimizers=keras.optimizers.RMSprop(learning_rate=1e-3),
                loss='sparse_categorical_crossentropy',
                metrics=['sparse_categorical_accuracy'])
  return model

### Many built-in optimizers, losses, and metrics are available

Optimizers: 
- `SGD()` (with or without momentum) 
- `RMSprop()` 
- `Adam()` etc.

Losses: 
 - `MeanSquaredError()` 
 - `KLDivergence()`
 - `CosineSimilarity()` .etc

Metrics:
- `AUC()`
- `Precision()`
- `Recall()` etc

### Custom losses
There are two ways to provide custom losses with Keras. The first example creates a function that accepts inputs `y_true` and `y_pred`. The following example shows a loss function that computes the averate distance between the real data and the predictions:

In [0]:
def basic_loss_function(y_true, y_pred):
  return tf.math.reduce_mean(y_true - y_pred)

model.compile(optimizers=keras.optimizers.Adam(),
              loss=basic_loss_function)

model.fit(x_train, y_train, batch_size=64, epochs=3)

If you need a loss function that takes in parameters besied `y_true` and `y_pred`, you can subclass the `tf.keras.losses.Loss` class and implement the following two methods:
- `__init__(self)`: Accept parameters to pass during the call of your loss function
- `call(self, y_true, y_tred)`: Use the target (`y_true`) and the model predictions (`y_pred`) to compute the model's loss

Parameters pass into `__init__()` can be used duing `call()` when calculating loss.

The following example shows how to implement a `WeightedCrossEntropy` loss function that calculates a `BiarnyCrossEntropy` loss, where the loss of a certain class or the whole function can be modified by a scalar

In [0]:
class WeightedBinaryCrossEntropy(keras.losses.Loss):
  """
  Args:
    pos_weights: Scalar to affect the positive labels of the loss function.
    weight: Scalar to affect the entirty of the loss function.
    from_logits: Whether to compute loss from logits or the probability.
    reduction: Type of tf.keras.losses.Reduction to apply to loss.
    name: Name of the loss fucntion.
  """
  def __init__(self, pos_weight, weight, from_logits=False,
               reduction=keras.losses.Reduction.AUTO,
               name='weighted_binary_crossentropy'):
    super(WeightedBinaryCrossEntropy, self).__init__(reduction=reduction,
                                                     name=name)
    self.pos_weight = pos_weight
    self.weight = weight
    self.from_logits = from_logits
  
  def call(self, y_true, y_pred):
    if not self.from_logits:
      # Manually calculate the weighted cross entropy.
      # Formula is q * z * - log(sigmoid(x)) + (1 - z) * -log(1 - sigmoid(x))
      # where z are labels, x is logits, and q is the weight.
      # Since the values passed are from sigmoid (assuming in this case)
      # sigmoid(x) will be replaced by y_pred

      # qz * -log(sigmoid(x)) 1e-6 is added as a epsilon to stop passing a zero
      # into the log
      x_1 = y_true * self.pos_weight * -tf.math.log(y_pred + 1e-6)

      # (1 - z) * -log(1 - sigmoid(x)). Epsilon is added to prevent passing a zero
      # into the log
      x_2 = (1 - y_true) * -tf.math.log(1 - y_pred + 1e-6)

      return tf.add(x_1, x_2) * self.weight
    # Use built in function
    return tf.nn.weighted_cross_entropy_with_logits(y_true, y_pred, self.pos_weight) * self.weight

In [0]:
model.compile(optimizer='adam',
              loss=WeightedBinaryCrossEntropy(0.5, 2))

In [0]:
model.fit(x_train, y_train, batch_size=64, epochs=3)

### Custom metrics
If you need a metric that isn't part of the API, you can easily create custom metrics by subclassing the `Metric` class. You will need to implement 4 methods:
- `__init__(self)`, in which you will create state variables for your metric.
- `update_state(self, y_true, y_pred, sample_weight=None)`, which uses the targets `y_true` and the model predictions `y_pred` to update the state variables.
- `result(self)`, which uses the state variables to compute the final results.
- `reset_states(self)`, which reinitializes the state of the metric

State update and results computation are kept separate (in `update_state()` and `result()`, respectively) because in some cases, results computation might be very expensive, and would only be done periodically.

Here's a simple example showing how to implement a `CategoricalTruePositives` metric, that counts how many samples where correctly classified as belonging to a give class:

In [0]:
class CategoricalTruePositives(keras.metrics.Metric):
  def __init__(self, name='categorical_true_positives', **kwargs):
    super(CategoricalTruePositives, self).__init__(name=name, **kwargs)
    self.true_positives = self.add_weight(name='tp', initializer='zeros')

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1, 1))
    values = tf.cast(y_true, 'int32') == tf.cast(y_pred, 'int32')
    values = tf.cast(values, 'float32')
    if sample_weight is not None:
      sample_weight = tf.cast(sample_weight, 'float32')
      values = tf.multiply(values, sample_weight)
    self.true_positives.assign_add(tf.reduce_sum(values))
  
  def result(self):
    return self.true_positives
  
  def reset_state(self):
    # The state of the metric will be reset at the start of each epoch.
    self.true_positives.assign(0.)


In [0]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[CategoricalTruePositives()])

In [0]:
model.fit(x_train, y_train,
          batch_size=64,
          epochs=3)

### Handling losses and metrics that don't fit the standard signiture
The overwhelming majority of losses and metrics can be computed from `y_true` and `y_pred`, where `y_pred` is an output of your model. But not all or them. For instance, a regularization loss may only require the activation of a layer (there are no target in this case), and this activation may not be a model output.

In such case, you can call `self.add_loss(loss_value)` from inside the `call` method of a custom layer. Here's a simple example that adds activity regularization (note that activity regularization is built-in in all Keras layers -- this layer is just for the sake of providing a concrete exmaple):

In [0]:
class ActivityRegularizationLayer(layers.Layer):
  def call(self, inputs):
    self.add_loss(tf.reduce_sum(inputs) * 0.1)
    return inputs # Pass-through layer.

inputs = keras.Input(shape=(784, ), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)

# Insert activity regularization as a layer
x = ActivityRegularizationLayer()(x)

x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

keras.Model(inputs, outputs)
model.compile(optimizer=keras.optimizers.RMSprop(1e-3),
              loss='sparse_categorical_crossentropy')

# Thedisplayed loss will be much higher than before
# due to the regularization component
model.fit(x_train, y_train,
          batch_size=64,
          epochs=1)

You can do the same for logging metric values:

In [0]:
class MetricLoggingLayer(layers.Layer):

  def call(self, inputs):
    # The `aggregation` argument defines
    # how to aggregate the per-batch values
    # over each epoch:
    # in this case we simply average them.
    self.add_metric(keras.backend.std(inputs),
                    name='std_of_activation',
                    aggregation='mean')
    return inputs  # Pass-through layer.

In [0]:
inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)

# Insert std logging as a layer.
x = MetricLoggingLayer()(x)

x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy')
model.fit(x_train, y_train,
          batch_size=64,
          epochs=1)

In the `Functional API`, you can also call `model.add_loss(loss_tensor)`, or `model.add_metric(metric_tensor, name, aggregation)`.


In [0]:
inputs = keras.Input(shape=(784,), name='digits')
x1 = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x2 = layers.Dense(64, activation='relu', name='dense_2')(x1)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x2)
model = keras.Model(inputs, outputs)

In [0]:
model.add_loss(tf.reduce_sum(x1) * 0.1)

In [0]:
model.add_metric(keras.backend.std(x1),
                 name='std_of_activation',
                 aggregation='mean')

In [0]:
model.compile(optimizer=keras.optimizers.RMSprop(1e-3),
              loss='sparse_categorical_crossentropy')

In [0]:
model.fit(x_train, y_train,
          batch_size=64,
          epochs=1)

### Automatically setting apart a validation holdout set
In the first end-to-end example you saw, we used the `validation_data` argument to pass a tuple of Numpy arrays `(x_val, y_val)` to the model for evaluating a validation loss and validation metrics at the end of each epoch.

Here's another option: the argument `validation_split` allows youto automatically reserve part of your training data for validation. The argument value represents the fraction of the data to be reserved for validation, so it should be set to a number higher than 0 and lower than 1.
For instance, `validation_split=0.2` means "use 20% of the data for validation", and `validation_split=0.6` means "use 60% of the data for validation."

The way the validation is computed is by taking the last x% samples of the arrays received by the fit call, before any shuffling.

In [0]:
model = get_compiled_model()
model.fit(x_train, y_train, batch_size=64, validation_split=0.2, epochs=3)

### Training & evaluation from tf.data Datasets
In the past few paragraphs, you've seen hot to handle losses, metrics, and optimizers, and you've seen how to use the `validation_data` and `validation_split` arguments in `fit`, when your data is passed as Numpy arrays.

Let's now take a look at the case where your data comes in the form of a tf.data.Dataset

The tf.data API is a set of utilities in TensorFlow 2.0 for loading and preprocessing data in a way that's fast and scalable.

You can pass a Dataset instances directly to the methods `fit()`, `evaluate()`, and `predict()`:



In [0]:
model = get_compiled_model()

# First, let's create a training Data instance.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle and slice the dataset.
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Now we get a test dataset.
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(64)

# Since the dataset already takes care of batching
# we don't pass a `batch_size` argument.
model.fit(train_dataset, epochs=3)

# Youcna also evaluate or predict on a dataset
print('\n# Evaluate')
model.evaluate(test_dataset)

Note that the Dataset is reset at the end of each epoch, so it can be reused of the next epoch.

If you want to run training only on a specific number of batches from this Dataset, you can pass the `steps_per_epoch` argument, which specifies how many training steps the model should run using this Dataset before moving on to the next epoch.

If you do this, the dataset is not reset at the end of each epoch, instead we just keep drawing the next batches. The dataset will eventually run out of data (unless it is an infinitely-looping dataset)

In [0]:
model = get_compiled_model()

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Only use the 100 batches per epoch (that is 64 * 100 samples)
model.fit(train_dataset.take(100), epochs=3)

### Using a validation dataset
You can pass a Dataset instance as the `validation_data` argument in `fit`:

In [0]:
model = get_compiled_model()

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

model.fit(train_dataset, epochs=3, validation_data=val_dataset)

At the end of each epoch, the model will iterate over the validation Dataset and compute the validation loss and validation metrics.

If you want to run validation only on a specific number of batches from this Dataset, you can pass the `validation_steps` argument, which specifies how many validation steps the model should run with the validation Dataset before interrupting validation and moving on to the next epoch:

In [0]:
model = get_compiled_model()

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

model.fit(train_dataset, epochs=3,
          # Only run validation using the first 10 batches of the dataset
          # using the `validation_steps` argument 
          validation_data=val_dataset, validation_steps=10)

Note that the validation Dataset will be reset after each use (so that you will always be evaluating on the same samples from epoch to epoch).

The argument `validation_split` (generating a holdout set from the training data) is not supported when training from Dataset objects, since this feature requires the ability to index the samples of the dataset, which is not possible in general without the Dataset API.

### Other input formats supported
Besides Numy arrays and TensorFlow Datasets, it's possible to train a Keras model using Pandas dataframes, or from Python generator that yield batches.

In general, we recommend that you use Numpy input data if your data is small and fits in memory, and Datasets otherwise.

### Using sample weighting and class weighting
Besides input and target data, it is possible to pass sample weights or class weights to a model when using `fit`:
- When training from Numpy data: via `sample_weight` and `class_weight` arguments.
- When training from Datasets: by having the Dataset return a tuple `(input_data, target_batch, sample_weight_batch)`.

A "sample weights" array is an array of numbers that specify how much weight each sample in a batch should have in computing the total loss. It is commonly used in imbalanced classification problems (the idea being to give more weight to rarely-seen classes). When the weights used are ones and zeros, the array can be used as a mask for the loss function (entirely discarding the contribution of certain samples to the total loss).

A "cass weights" dict is a more specific instance of the same concept: it maps class indices to the sample weight that should be used for samples belonging to this class. For instance, if class "0" is twice less represented than class "1" in your data, you could use `class_weight={0: 1., 1: 0.5}`.

In [0]:
import numpy as np

class_weight = {0: 1., 1: 1., 2: 1., 3: 1., 4: 1.,
                # Set weight "2" for class "5"
                # making this class 2x more important
                5: 2., 6: 1., 7: 1., 8: 1., 9: 1.}
print('Fit with class weight')
model.fit(x_train, y_train,
          class_weight=class_weight,
          batch_size=64,
          epochs=4)

# Here's the sample example using `sample_weight` instead:
sample_weight = np.ones(shape=(len(y_train),))
sample_weight[y_train == 5] = 2.
print('\nFit with sample weight')
model = get_compiled_model()
model.fit(x_train, y_train,
          sample_weight=sample_weight,
          batch_size=64,
          epochs=4)

In [0]:
sample_weight = np.ones(shape=(len(y_train), ))
sample_weight[y_train==5] = 2.

# Create a Dataset that includes sample weights
# (3rd element in the return tuple).
train_dataset = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train, sample_weight))
  
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)
model = get_compiled_model()
model.fit(train_dataset, epochs=3)

### Passing data to multi-input, multi-output models
Consider the following model, which has an image input of shape`(32, 32, 3)` (that's `(height, width, channels)`) and a timeseries input of shape `(None, 10)` (that's `(timesteps, features)`). Our model will have two outputs computed from the combination of these inputs: a "score" (of shape`(1,)`) and probability distribution over 5 classes (of shape `(10, )).

In [0]:
from tensorflow import keras
from tensorflow.keras import layers

image_input = keras.Input(shape=(32, 32, 3), name='img_input')
timeseries_input = keras.Input(shape=(None, 10), name='ts_input')

x1 = layers.Conv2D(3, 3)(image_input)
x1 = layers.GlobalMaxPooling2D()(x1)

x2 = layers.Conv1D(3, 3)(timeseries_input)
x2 = layers.GlobalMaxPooling1D()(x2)

x = layers.concatenate([x1, x2])

score_output = layers.Dense(1, name='score_output')(x)
class_output = layers.Dense(5, activation='softmax', name='class_output')(x)

model = keras.Model(inputs=[image_input, timeseries_input],
                    outputs=[score_output, class_output])

In [0]:
keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

At compilation time, we can specify different losses to different outputs, by passing loss functions as a list:

In [0]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[keras.losses.MeanSquaredError(),
          keras.losses.CategoricalCrossentropy()]
)

If we only passed a single loss function to the model, the same loss function would be applied to every output, which is not appropriate here.

In [0]:
# Since we gave names to our output layers,
# we could also specify per-output losses and metrics via a dict:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={'score_output': keras.losses.MeanSquaredError(),
          'class_output': keras.losses.CategoricalCrossentropy()},
    metrics={
        'score_output': [keras.metrics.MeanAbsolutePercentageError(),
                         keras.metrics.MeanAbsoluteError()],
        'class_output': [keras.metrics.CategoricalAccuracy()]})

We recommend the use of explicit names and dics if you have more than 2 outputs.

It's possible to give different weights to different output-specific losses(for instance, one might wish to privilege the "score" loss in our example, by giving to 2x the importance of the class loss), using `loss_weight` argument:

In [0]:
model.compile(
    optimizer='rmsprop',
    loss={'score_output': 'mse',
          'class_output': 'categorical_crossentropy'},
    metrics={'score_output': ['mae', keras.metrics.MeanAbsolutePercentageError()],
             'class_output': 'categorical_accuracy'},
    loss_weight={'score_output': 2., 'class_output': 1.})

You could also chose not to compute a loss for certain outputs, if these outputs meant for prediction but not for training


In [0]:
# List loss version
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[None, keras.losses.CategoricalCrossentropy()])

# Or dict loss version
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={'class_output': keras.losses.CategoricalCrossentropy()})

Passing data to a multi-input or multi-output model in `fit` works in a similar way as specifying a loss function in `compile`: you can pass lists of Numpy arrays (with 1:1 mapping to the outputs that received a loss function) or dicts mapping output names to Numpy arrays of training data.

In [0]:
model.compile(
    optimizer='rmsprop',
    loss=['mse', 'categorical_crossentropy'])

# Generate dummy Numpy data
img_data = np.random.random_sample(size=(100, 32, 32, 3))
ts_data = np.random.random_sample(size=(100, 20, 10))
score_targets = np.random.random_sample(size=(100, 1))
class_targets = np.random.random_sample(size=(100, 5))

model.fit([img_data, ts_data], [score_targets, class_targets],
          batch_size=32,
          epochs=3)

model.fit({'img_input': img_data, 'ts_input': ts_data},
          {'score_output': score_targets, 'class_output': class_targets},
          batch_size=32,
          epochs=3)

Notice that we give label to output layer.

In [0]:
# Here's the Dataset use case: similarly as what we did for Numpy arrays,
# the Dataset should return a tuple of dicts.
train_dataset = tf.data.Dataset.from_tensor_slices(
    ({'img_input': img_data, 'ts_input': ts_data},
     {'score_output': score_targets, 'class_output': class_targets}))
train_dataset = train_dataset.shuffle(1024).batch(64)

model.fit(train_dataset, epochs=3)

### Using callbacks
Callbacks in Keras are objects that are called at different point during training (at the start of an epoch, at the end of btch, at the end of an epoch, etc) and which can be used to implement behavioirs such as:
- Doing validation at different points during training (beyond the built-in per-epoch validation
- Checkpointing the model at regular intervals or when it exceeds a certain accuracy threshold
- Changing the learning rate of the model when training seems to be plateauing
- Doing fine-tuning of the top layers when training seems to be plateauing
- Sending email or instance message notifications when training ends or where a certain performance threshold is exceeded
- Etc.

In [0]:
# Callbacks can be passed as a list to your call to `fit`
model = get_compiled_model()

callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=2,
        verbose=1
    )
]
model.fit(x_train, y_train,
          epochs=20,
          batch_size=64,
          callbacks=callbacks,
          validation_split=0.2)

**Many built-in callbacks are available**
- `ModelCheckpoint`: periodically save the model
- `EarlyStopping`: Stop training when training is no longer improving the validation metrics.
- `TensorBoard`: periodically write model logs that can be visualized in TensorBoard
- `CSVLogger`: streams loss and metrics data to a CSV file
- etc.

### Writing your own callback
You can create a custom callback by extending the base class keras.
callbacks.Callback. A callback has access to its associated model through the class property `self.model`

Here's a simple example saving a list of per-batch loss values during training:

In [0]:
class LossHistory(keras.callbacks.Callback):
  def on_train_begin(self, logs):
    self.losses = []
  
  def on_batch_end(self, batch, logs):
    self.losses.append(logs.get('loss'))

## Checkpointing models
When you're training model on relatively large datasets, it's crucial to save checkpoints of your model at frequent intervals.

In [0]:
model = get_compiled_model()

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='mymodel_{epoch}.h5',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

model.fit(x_train, y_train,
          epochs=3,
          batch_size=64,
          callbacks=callbacks,
          validation_split=0.2)

Using learning rate schedules
A common pattern when training deep learning models is to gradually reduce the learning as training progresses. This is generally known as "learning rate decay".

The learning decay schedule could be static (fixed in advance, as a function of the current epoch or the current batch index), or dynamic (responding to the current behavior of the model, in particular the validation loss).

### passing a schedule to an optimizer
You can easily use a static learning rate decay schedule by passing a schedule object as the `learning_rate` argument in your optimizer:

In [0]:
initial_learning_rate = 0.1
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)

optimizer = keras.optimizers.RMSprop(learning_rate=lr_schedule)

Several built-in schedules are available: ExponentialDecay, PiecewiseConstantDecay, PolynomialDecay, and InverseTimeDecay.

### using callbacks to implement a dynamic learning rate schedule
A dynamic learning rate schedule (for instance, decreasing the learning rate when the validation loss is no longer improving) cannot be achieved with these schedule objects since the optimizer does not have access to validation metrics.

However, callbacks do have access to all metrics, including validation metrics! You can thus achieve this pattern by using a callbacks that modifies the current learning rate on the optimizer. In fact, this is even built-in as the `ReduceLROnPlateau` callback

### Visualizing loss and metrics during training
The best way to keep an eye on your model during training is use `TensorBoard`, a browser-based application that you can run locally that provides you with:
- Live plots of the loss and metrics for training and evaluation
- (optionally) Visualizations of the histograms of your layer activations
- (optionally) 3D visualizitaions of the embedding spaces learned by your `Embedding` layers

### Using the TensorBoard callback
The easiest way to use TensorBoard with a Keras model and the `fit` method is the `TensorBoard` callback.
In the simplest case, just specify where you want the callback to write logs, and you're good to go:

In [0]:
keras.callbacks.TensorBoard(
  log_dir='/full_path_to_your_logs',
  histogram_freq=0,  # How often to log histogram visualizations
  embeddings_freq=0,  # How often to log embedding visualizations
  update_freq='epoch')  # How often to write logs (default: once per epoch)

## Writing your own training & evaluation loops from scratch

If you want lower-level over your training & evaluation loops than what `fit()` and `evaluate()` provide, you should write your own. 

### Using the GradientTape:

Calling a model inside a `GradientTape` scope enables you to retrieve the gradients of the trainable weights of the layer with respect to a loss value. Using an optimizer instance, you can use these gradients to update these gradients to update these variables (which you can retrieve using `model.trainable_weights`).

In [0]:
inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy()

# Prepare the training dataset.
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Iterate over epochs.
for epoch in range(3):
  print('Start of epoch %d' % (epoch, ))

  # Iterate over the batches of the dataset.
  for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

    # Open a GradientTape to record the operations run
    # during the forward pass, which enable autodifferentiation.
    with tf.GradientTape() as tape:

      # Run the forward pass of the layer.
      # The operations that the layer applies
      # to its inputs are going to be recorded
      # on the GradientTape
      logits = model(x_batch_train) # Logits for this minibatch

      # Compute the loss value for this minibatch
      loss_value = loss_fn(y_batch_train, logits)
    
    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss.
    grads = tape.gradient(loss_value, model.trainable_weights)

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    # Log every 200 batches
    if step % 200 == 0:
      print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))
      print('Seen so far: %s samples' % ((step + 1) * 64))

### Low-level handling of metrics

Let's add metrics to the mix. You can readily reuse the built-in metrics (or custom ones you wrote) in such training loops written from scratch.
- Instantiate the metric at the start of the loop
- Call `metric.update_state()` after each batch
- Call `metric.result()` when you need to display the current value of the metric
- Call `metric.reset_states()` when you need to clear the state of the metric(typically at the end of an epoch).


In [0]:
# Compute `SparseCategoricalAccuracy` on validation data at the end of each epcoh

# Get model
inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

optimizer = keras.optimizers.SGD(learning_rate=1e-3)
loss_nf = keras.losses.SparseCategoricalCrossentropy()

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

# Prepare the training dataset.
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

In [0]:
for epoch in range(3):
  print('Start of epoch %d' % (epoch, ))

  # Iterate over the batches of the dataset
  for stpe, (x_batch_train, y_batch_train) in enumerate(train_dataset):
    with tf.GradientTape() as tape:
      logits = model(x_batch_train)
      loss_value = loss_fn(y_batch_train, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    # Update training metric.
    train_acc_metric(y_batch_train, logits)

    # Log every 200 batches.
    if step % 200 == 0:
      print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))
      print('Seen so far: %s samples' % ((step + 1) * 64))

  # Display metrics at the end of each epoch.
  train_acc = train_acc_metric.result()
  print('Training acc over epoch: %s' % (float(train_acc),))
  # Reset training metrics at the end of each epoch
  train_acc_metric.reset_states()

  # Run a validation loop at the end of each epoch.
  for x_batch_val, y_batch_val in val_dataset:
    val_logits = model(x_batch_val)
    # Update val metrics
    val_acc_metric(y_batch_val, val_logits)
  val_acc = val_acc_metric.result()
  val_acc_metric.reset_states()
  print('Validation acc: %s' % (float(val_acc), ))


### Low-level handling of extra losses
You saw in the previous section that it is possible for regularization losses to be added by a layer by calling `self.add_loss(value)` in the `call` method.

In the general case, you will want to take these losses into account in your custom training loops (unless you've written the model yourself and you already know that it creates no such losses

In [0]:
class ActivityRegularizationLayer(layers.Layer):
  def call(self, inputs):
    self.add_loss(1e-2 * tf.reduce_sum(inputs))
    return inputs

In [0]:
inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
# Insert activity regularization as a layer
x = ActivityRegularizationLayer()(x)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [0]:
logits = model(x_train[:64])

the losses it creates during the forward pass are added to the `model.losses` attribute:

In [0]:
logits = model(x_train[:64])
print(model.losses)

In [0]:
logits[0]

The tracked losses are first cleared at the start of the model `__call__`, so you will only see the losses created during this one forward pass. For instance, calling the model repeatedly and then querying `losses` only displays the latest losses, created during the last call:


In [0]:
logits = model(x_train[:64])
logits = model(x_train[64: 128])
logits = model(x_train[128: 192])
print(model.losses)

To take these losses into account during training, all you have to do is to modify your training loop to add `sum(model.losses)` to your total loss:

In [0]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3)

for epoch in range(3):
  print('Start of epoch %d' % (epoch, ))

  for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
    with tf.GradientTape() as tape:
      logits = model(x_batch_train)
      loss_value = loss_fn(y_batch_train, logits)

      loss_value += sum(model.losses)
    
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    if stpe % 200 == 0:
      print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))
      print('Seen so far: %s samples' % ((step + 1) * 64))